In [663]:
from collections import OrderedDict


def calculate_value(value, unit):
    multipliers = {"u": 1e-6, "m": 1e-3, "": 1, "k": 1e3, "M": 1e6}
    return value * multipliers[unit]

def find_n_cycles(graph, start, current, visited, path, cycles, cycles_to_find):
    # Помечаем текущую вершину как посещенную
    visited[current] = True
    # Добавляем текущую вершину в путь
    path.append(current)

    # Проверяем, есть ли ребро от текущей вершины к начальной (формирование цикла)
    if start in graph[current]:
        cycles.append(path[:])  # Добавляем цикл в список циклов
        if len(cycles) >= cycles_to_find:
            return cycles  # Если достигнуто нужное количество циклов, возвращаем список циклов

    # Рекурсивно обходим соседей текущей вершины
    for neighbor in graph[current]:
        if not visited[neighbor]:
            cycles = find_n_cycles(
                graph, start, neighbor, visited, path, cycles, cycles_to_find
            )
            if len(cycles) >= cycles_to_find:
                return cycles  # Если достигнуто нужное количество циклов, возвращаем список циклов

    # Убираем текущую вершину из пути и помечаем как непосещенную для других циклов
    path.pop()
    visited[current] = False

    return cycles

def sort_lists_in_dict(input_dict):
    sorted_dict = {
        key: sorted(value, reverse=True) for key, value in input_dict.items()
    }
    return sorted_dict

def format_number_dict(input_dict):
    formatted_dict = OrderedDict()
    for key, value in sorted(input_dict.items()):
        if value >= 1e6:
            formatted_dict[key] = str(round(value / 1e6)) + "M"
        elif value >= 1e3:
            formatted_dict[key] = str(round(value / 1e3)) + "k"
        elif value >= 1:
            formatted_dict[key] = str(round(value))
        elif value >= 1e-3:
            formatted_dict[key] = str(round(value / 1e-3)) + "m"
        elif value >= 1e-6:
            formatted_dict[key] = str(round(value / 1e-6)) + "u"
        else:
            formatted_dict[key] = "0"
    return formatted_dict

N = int(input())
components = {}

for _ in range(N):
    component, node1, node2, value = input().split()
    value_number = (
        float(value[:-1]) if not value[-1].isdigit() else float(value)
    )  # Учитываем случай, когда буква отсутствует
    value_unit = (
        value[-1] if value[-1].isalpha() else ""
    )  # Учитываем случай, когда буква отсутствует
    components[component] = (
        node1,
        node2,
        calculate_value(value_number, value_unit),
    )

"""
Создает словарь с точками
"""
points = {}

for key, value in components.items():
    unique_elements = set(
        value[:2]
    )  # Получаем уникальные элементы из первых двух позиций кортежа
    for element in unique_elements:
        if element not in points:
            points[element] = []
        points[element].append(key)  # Добавляем ключ в список значений нового словаря

"""
Создает список с висячими резисторами
"""
without_current = []
R_to_remove = []
key_to_remove = []

for key, value in points.items():
    if len(value) == 1:
        without_current.append(value[0])
        R_to_remove.append(value[0])
        key_to_remove.append(key)

for key in points.keys():
    for r in R_to_remove:
        if r in points[key]:
            points[key].remove(r)

for key in key_to_remove:
    points.pop(key)
"""
Создает список с узлами
"""
nodes = []

for key, value in points.items():
    if len(value) >= 3:
        nodes.append(key)

"""
Создает словарь с узлами и соседними узлами
"""
only_points = {key: [] for key in points if points[key]}

for key, edges in points.items():
    for edge in edges:
        for index, value in points.items():
            if edge in value and key != index:
                only_points[key].append(index)

"""
Находит количество независимых ветвей
"""
branches = 0
for node, neighbors in points.items():
    if len(neighbors) >= 3:
        branches += len(neighbors)
branches /= 2

"""
Создает список из списков маршрутов (через точки)
"""
n = branches - len(nodes) + 1  # Находим количество циклов для поиска

# Список для хранения найденных циклов
found_cycles = []

# Начинаем обход графа для поиска указанного количества циклов, начиная с указанной вершины
visited = {key: False for key in only_points}
if len(nodes) > 0:
    start = nodes[0]
else:
    start = 0
n_cycles = find_n_cycles(only_points, start, start, visited, [], found_cycles, n)

for l in n_cycles:
    l.append(start)

"""
Создаем словарь пар точек, где аргументы - цепи
"""
pairs = {}

# Проходимся по ключам и значениям словаря points
for key_i, value_i in points.items():
    for key_j, value_j in points.items():
        if (
            key_i < key_j
        ):  # Чтобы избежать повторений (например, если проверили (1, 2), то не проверять (2, 1))
            intersection = set(value_i) & set(value_j)
            if intersection:
                pairs[f"{key_i} {key_j}"] = list(intersection)

# Получение отсортированного словаря
pairs = sort_lists_in_dict(pairs)

"""
Метод контурных токов
"""
R_and_I = dict()
for cycles in n_cycles:
    R = 0
    V = 0
    k = 0
    use_resistors = []
    for i in range(len(cycles) - 1):
        if not (s == f"{cycles[i]} {cycles[i+1]}" or s == f"{cycles[i+1]} {cycles[i]}"):
            k = 0
        s = f"{cycles[i]} {cycles[i+1]}"
        if s in pairs:
            if pairs[s][k][0] == "R":
                R += components[pairs[s][k]][2]
                use_resistors.append(pairs[s][k])
                if len(pairs[s]) > 1:
                    pairs[s].remove(pairs[s][k])
            else:
                if f"{components[pairs[s][k]][1]} {components[pairs[s][k]][0]}" == s:
                    V += components[pairs[s][k]][2]
                    if len(pairs[s]) > 1:
                        k += 1
                else:
                    V -= components[pairs[s][k]][2]
                    if len(pairs[s]) > 1:
                        k += 1
        else:
            s = f"{cycles[i+1]} {cycles[i]}"
            if pairs[s][k][0] == "R":
                R += components[pairs[s][k]][2]
                use_resistors.append(pairs[s][k])
                if len(pairs[s]) > 1:
                    pairs[s].remove(pairs[s][k])
            else:
                if f"{components[pairs[s][k]][1]} {components[pairs[s][k]][0]}" != s:
                    V += components[pairs[s][k]][2]
                    if len(pairs[s]) > 1:
                        k += 1
                else:
                    V -= components[pairs[s][k]][2]
                    if len(pairs[s]) > 1:
                        k += 1
    I = V / R
    for r in use_resistors:
        if r in R_and_I:
            R_and_I[r] += I
        else:
            R_and_I[r] = I

R_and_P = dict()
for resistance, current in R_and_I.items():
    R_and_P[resistance] = components[resistance][2] * current**2

for r in R_to_remove:
    R_and_P[r] = 0

output_dict = format_number_dict(R_and_P)
for key, value in output_dict.items():
    print(f"{key} {value}")

R1 102m
